# Projeto 2: Teste AB

### <font color='blue'> Alexandre Young - B</font> ###
### <font color='blue'> Jean Walper - B</font> ###
### <font color='blue'> Pedro de la Peña - B</font> ###
### <font color='blue'> Sabrina Sampaio Simão - A</font> ###

## Introdução:

O nosso projeto consiste em propor um teste AB para um site, aplicá-lo e analisar seus resultados.
    
Um teste AB é um tipo de teste usado para analisar os efeitos que uma caracteristica diferente tem sobre uma determinada tarefa. O utilizaremos para estudar os efeitos que a disposição de links tem na navegação de um site.
  
Como exemplo real utilizamos um site que acreditamos que se beneficiaria dessa mudança, o portfólio de projetos (primeiro projeto de Co-Design) de um dos integrantes do grupo. No modelo original do site (A), o posicionamento dos links para as paginas de projeto parecia causar uma certa confusão sobre quais itens eram links "clicáveis" ao estar abaixo de itens que não eram. Para confirmar essa hipótese construímos uma versão modificada (B) onde acreditamos que a disposição dos links fosse mais intuitiva.

O motivo para essa mudança veio do próprio feedback dos avaliadores do portfolio, portanto há uma base para pensar que tal alteração será efetiva para o tempo de uso do nosso site. Na versão B, imaginamos que o usuário vai encontrar os links dos trabalhos muito mais rápido!

Fizemos, então, duas versões diferentes do site e a colocamos no firebase hosting. O usuário que entrasse no site teria 50% de chance de cair em uma das versões A ou B.

Encontramos 79 candidatos dispostos a nos ajudar no teste, sem conhecer ou ser exposto à outra versão do site e sem nenhuma indicação de que caminho seguir, pedimos que cada candidato encontrasse a página do "Jogo de Computador" no portfolio. Para os 42 que navegaram pela versão A do site e os 37 pela a versão B procuramos medir apenas o tempo que levaram, em segundos para encontrar a página específica.
 
Tentamos nos certificar que eles não conheciam o site previamente, eram usuários que não tinham conhecimento do projeto de co design do Insper, e também não fossem muito leigos em informática, para não ficarem perdidos na tarefa.

In [1]:
import pandas as pd
import json
import itertools
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
%matplotlib inline

#### Leitura do arquivo

In [2]:
arquivo = open("exportNINJA.json")
linhas = arquivo.readlines()
texto = "".join(linhas)
dados_fire = json.loads(texto)

#### Análise do atributo logs

In [3]:
for k,v in dados_fire["logs"].items():
    # print(k,v) # habilite para imprimir as chaves de mais alto nível
    for chave, valor in v.items():
        # print(chave, valor) # habilite para imprimir as chaves inferiores
        pass # desabilite quando reabilitar a linha de cima

#### Convertendo para um dataframe

In [4]:
lista_dicionarios = []

for k,v in dados_fire["logs"].items():
    lista_dicionarios.append(v)

In [5]:
dataframe = pd.DataFrame.from_records(lista_dicionarios)

In [6]:
#dataframe (desmarcar para ver o dataframe)

#### Convertendo para um DataFrame de delta_t

In [7]:
agrupado = dataframe.groupby(by=["user_id", "versao"])

In [8]:
users = set(dataframe.user_id) # We convert it to a set so that each user appear only once

In [9]:
users = list(users)

In [10]:
resultados = []

for u in users:
    for versao in ["A", "B"]:
        if (u, versao) in agrupado.groups.keys(): # Check if a specific pair user/type of test happened
            grupo1 =  agrupado.get_group((u, versao))
            grupo1 = grupo1.sort_values(by="tempo") # Para evitar que inícios sem fim gerem tempos negativos
            
            if len(grupo1[grupo1.tipo=="fim"])>= 1 and len(grupo1[grupo1.tipo=="inicio"]) >= 1:
                delta_t = grupo1[grupo1.tipo=="fim"].iloc[-1].tempo - grupo1[grupo1.tipo=="inicio"].iloc[0].tempo
                resultados.append({"user_id":u, "delta_t":delta_t, "versao":versao})

In [11]:
#grupo1.sort_values(by="tempo")

In [12]:
dataframe_resumo = pd.DataFrame.from_records(resultados)

In [13]:
dataframe_resumo.delta_t/=1000 # Convertendo milissegundos em segundos

In [14]:
dfA= dataframe_resumo[dataframe_resumo.versao == "A"]
dfB= dataframe_resumo[dataframe_resumo.versao == "B"]


print("Numero da população de A e B, respectivamente")
print(dfA.shape[0],dfB.shape[0])

#dataframe_resumo (desmarcar para ver nossa tabela)

Numero da população de A e B, respectivamente
42 37


# <color='m'> Teste de hipótese! </color>
### Usando teste t, pois em nossa amostra, n < 200

Definimos a hipótese nula como o tempo médio gasto na versão A do site equivalendo ao tempo médio gasto na versão B (H0:μa=μb) e a hipótese alternativa como o valor médio de tempo gasto na versão A sendo maior que o tempo gasto na versão B (H1:μa > μb). O nível de significância inicial foi de 5%, mas resolvemos analisar para 1% e 10% também.
    
Devido ao fato de não possuirmos muitas amostras, realizamos o teste t-student em vez de um teste z. Os parâmetros do teste estão definidos no código a seguir, sendo T a média padronizada e V o grau de liberdade:

In [15]:
alpha= 0.05 #Nivel de significancia 10%
na= dfA.shape[0]
nb = dfB.shape[0]
D = 0 #Pois adimitimos H0: uma media igual a outra

#Hipótese:
    #H0: μa = μb
    #H1: μa =/= μb
    
μa= dfA.delta_t.mean()
μb= dfB.delta_t.mean()

Sa= dfA.delta_t.std()
Sb= dfB.delta_t.std()

sna = ((Sa**2)/na)
snb = ((Sb**2)/nb)
spa = (sna**2)/(na-1)
spb = (snb**2)/(nb-1)

#LES FORMULAS(3B, magalhães e lima)

T=(D-(μa - μb))/((sna+snb)**(0.5))
v= ((sna + snb)**2)/(spa+spb)

### Na próxima célula, analisaremos a posição do valor-p em relação ao alpha

In [16]:
pvalue = stats.t.sf(T,df=v)
print(pvalue)
print()

alpha=0.1
if(pvalue> 1-alpha):
    print("Com nosso nível de significância alpha= 10%, nosso teste T REJEITA a hipótese nula")
else:
    print("Com nosso nível de significância alpha= 10%, nosso teste T NÃO REJEITA a hipótese nula")
    
alpha=0.05
if(pvalue> 1-alpha):
    print("Com nosso nível de significância alpha= 5%, nosso teste T REJEITA a hipótese nula")
else:
    print("Com nosso nível de significância alpha= 5%, nosso teste T NÃO REJEITA a hipótese nula")
    
alpha=0.01
if(pvalue> 1-alpha):
    print("Com nosso nível de significância alpha= 1%, nosso teste T REJEITA a hipótese nula")
else:
    print("Com nosso nível de significância alpha= 1%, nosso teste T NÃO REJEITA a hipótese nula")

0.964135517461

Com nosso nível de significância alpha= 10%, nosso teste T REJEITA a hipótese nula
Com nosso nível de significância alpha= 5%, nosso teste T REJEITA a hipótese nula
Com nosso nível de significância alpha= 1%, nosso teste T NÃO REJEITA a hipótese nula


## Conclusão: ###

A validade do nosso teste T depende do nosso nível de confiança alpha em que trabalhamos, sendo alpha a probabilidade de que nosso teste não ser confiável. Não é possível, no entanto, assumir um nivel total de confiabilidade (alpha de 0%). Isto apenas seria possível caso existisse um espaço amostral que representasse com absoluta certeza a população total. 
  	
Queremos idealmente estudar nosso teste dentro de níveis altos de confiabilidade para uma análise final mais significativa. A mudança do valor de alpha pode confirmar ou rejeitar a hipotese nula, portanto decidimos fazer o teste com os tres valores mais comuns de alpha: 1%, 5% e 10%.
  	
Após os teste, percebemos que a hipotese nula é rejeitada com um nivel de significancia de 10% e 5%, indicando que há uma diferença consideravel na media do tempo nescessario para executar a tarefa do site em suas diferentes versões, e
mostrando que a posição dos links dos projetos influenciou nesse tempo. Porém, quando diminuimos o alpha (nivel de significancia) para 1%, a hipotese nula passou a não ser rejeitada, indicando que não há diferença entre as duas versões, em quesitos de tempo gasto para achar o projeto alvo. Isto se deve ao fato de nossa população não ser perfeita a ponto de conseguirmos uma amostra maior e mais dedicada. De qualquer forma, nosso p-value foi de menos de 4%, razoavelmente baixo, assim podemos realmente duvidar da hipótese nula.
  	
Concluímos que, ao analisar os dados coletados, com nível de confiabilidade de 95%, a nossa alteração na disposição dos itens do site beneficiou significativamente a navegabilidade, facilitando para os usuários encontrar as informações que procuram. 
   